In [1]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st 
from citipy import citipy
import gmaps
import os

# Import API key
from api_keys import gkey

In [2]:
# Store Part I results into DataFrame
path = "../output_data/city_weather_data.csv"
weather_df = pd.read_csv(path)
weather_df = weather_df.drop(['Unnamed: 0'], axis=1)

weather_df

,City,Country,Longitude,Latitude,Weather,Temp,Temp (Max),Temp (Min),Humidity,Cloudiness,Wind Speed
0,Tuatapere,NZ,167.6833,-46.1333,broken clouds,43.83,43.83,43.83,78,70,2.37
1,Fort Saint James,CA,-124.2500,54.4333,scattered clouds,80.60,80.60,80.60,23,28,8.81
2,Cockburn Town,TC,-71.1419,21.4612,few clouds,81.05,81.05,81.05,75,18,15.50
3,Tiksi,RU,128.8694,71.6872,overcast clouds,43.93,43.93,43.93,68,91,20.49
4,Bijapur,IN,75.7000,16.8333,overcast clouds,71.47,71.47,71.47,89,100,20.07
...,...,...,...,...,...,...,...,...,...,...,...
686,Kargasok,RU,80.8572,59.0556,broken clouds,60.60,60.60,60.60,95,54,12.01
687,Makkah al Mukarramah,SA,39.8261,21.4267,scattered clouds,88.66,88.66,88.66,26,43,5.95
688,Yar-Sale,RU,70.8333,66.8333,clear sky,43.79,43.79,43.79,65,6,5.82
689,Zhicheng,CN,111.5047,30.2956,light rain,80.60,80.60,80.60,82,62,3.29


In [3]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

# Create a list containing coordinates
lat = weather_df['Latitude'].tolist()
lng = weather_df['Longitude'].tolist()

coordinates = []
for i in range(0, 691):
    coordinates.append((lat[i], lng[i]))


In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(map_type='ROADMAP', layout=figure_layout)

# Assign the marker layer to a variable
heatmap = gmaps.heatmap_layer(coordinates, weights=weather_df['Humidity'])

# Add the heatmap layer to the map
fig.add_layer(heatmap)

# Generate figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
ideal_df = weather_df[(weather_df['Temp (Max)']<75) & (weather_df['Temp (Min)']>60) & (weather_df['Humidity']>30) 
                     & (weather_df['Wind Speed']<10) & (weather_df['Cloudiness']==0)].reset_index()
ideal_df

,index,City,Country,Longitude,Latitude,Weather,Temp,Temp (Max),Temp (Min),Humidity,Cloudiness,Wind Speed
0,31,Rikitea,PF,-134.9692,-23.1203,clear sky,73.11,73.11,73.11,72,0,9.13
1,90,Pires do Rio,BR,-48.2794,-17.2997,clear sky,67.46,67.46,67.46,39,0,3.89
2,129,Dzaoudzi,YT,45.2699,-12.7887,clear sky,73.29,73.29,73.29,88,0,5.75
3,307,Vezirköprü,TR,35.4547,41.1436,clear sky,70.29,70.29,70.29,89,0,2.24
4,337,Tortolì,IT,9.6572,39.9256,clear sky,70.99,70.99,70.99,70,0,6.62
5,416,Saint-Pierre,RE,55.4781,-21.3393,light rain,66.07,67.64,65.88,77,0,6.91
6,551,Starovelichkovskaya,RU,38.7326,45.4288,clear sky,68.04,70.00,68.02,43,0,8.61
7,592,Kropotkin,RU,40.5756,45.4375,clear sky,70.34,70.34,70.34,39,0,7.85
8,602,Abhā,SA,42.5053,18.2164,clear sky,66.45,66.45,66.45,77,0,0.00
9,644,Teplyk,UA,29.7450,48.6657,clear sky,71.76,71.76,71.76,69,0,5.53


In [6]:
# Use Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# Create a list containing coordinates
ideal_lat = ideal_df['Latitude'].tolist()
ideal_lng = ideal_df['Longitude'].tolist()
ideal_cities = ideal_df['City'].tolist()
ideal_countries = ideal_df['Country'].tolist()

ideal_coord = []
for i in range(0, 11):
    ideal_coord.append((ideal_lat[i], ideal_lng[i]))
    
hotel_names = []
place_names = []

for coord in ideal_coord:
# geocoordinates
    target_coordinates = coord
    target_radius = 5000
    target_type = "lodging"

# set up a parameters dictionary
    params = {
        "location": f'{target_coordinates[0]},{target_coordinates[1]}',
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using parameters dictionary
    response = requests.get(base_url, params=params)

    places_data = response.json()

    hotel_names.append(places_data["results"][0]["name"])
    place_names.append(places_data["results"][0]["vicinity"])

print(place_names)
print(hotel_names)

['Rikitea', 'Avenida Lino Sampaio, 5 - Centro, Pires do Rio', 'Jetée Issoufali n.n., Dzaoudzi 97610', 'Fazıl Ahmet Paşa, 121. Sok. No:19', "Via Porto Frailis, Arbatax, Tortoli'", '21 Rue Francois Isautier', 'Ulitsa Gorodskaya, 156, Starovelichkovskaya', 'Ulitsa Bazarnaya, 50, Kropotkin', 'King Saud Street, Al Aziziah District, Abha', 'Незалежності вул. 36, Teplyk', 'p. Severnyi-Pervyi, ul.Berezovaia, 1/21, Belgorod']
['People ThankYou', 'Hotel Goiano - Pires do Rio', 'Le Rocher', 'Vezirkopru Ogretmenevi', 'Hotel La Bitta', 'Lindsey Hôtel', 'Yubileynaya Hotel', 'Hotel Europe Kropotkin', 'Abha Hotel', 'Pp Hotelʹnyy Kompleks Pryvat-Otelʹ', 'Full House North Mini-Hotel']


In [7]:
hotel_df = ideal_df.copy()
hotel_df["Hotel"]=hotel_names
hotel_df

,index,City,Country,Longitude,Latitude,Weather,Temp,Temp (Max),Temp (Min),Humidity,Cloudiness,Wind Speed,Hotel
0,31,Rikitea,PF,-134.9692,-23.1203,clear sky,73.11,73.11,73.11,72,0,9.13,People ThankYou
1,90,Pires do Rio,BR,-48.2794,-17.2997,clear sky,67.46,67.46,67.46,39,0,3.89,Hotel Goiano - Pires do Rio
2,129,Dzaoudzi,YT,45.2699,-12.7887,clear sky,73.29,73.29,73.29,88,0,5.75,Le Rocher
3,307,Vezirköprü,TR,35.4547,41.1436,clear sky,70.29,70.29,70.29,89,0,2.24,Vezirkopru Ogretmenevi
4,337,Tortolì,IT,9.6572,39.9256,clear sky,70.99,70.99,70.99,70,0,6.62,Hotel La Bitta
5,416,Saint-Pierre,RE,55.4781,-21.3393,light rain,66.07,67.64,65.88,77,0,6.91,Lindsey Hôtel
6,551,Starovelichkovskaya,RU,38.7326,45.4288,clear sky,68.04,70.00,68.02,43,0,8.61,Yubileynaya Hotel
7,592,Kropotkin,RU,40.5756,45.4375,clear sky,70.34,70.34,70.34,39,0,7.85,Hotel Europe Kropotkin
8,602,Abhā,SA,42.5053,18.2164,clear sky,66.45,66.45,66.45,77,0,0.00,Abha Hotel
9,644,Teplyk,UA,29.7450,48.6657,clear sky,71.76,71.76,71.76,69,0,5.53,Pp Hotelʹnyy Kompleks Pryvat-Otelʹ


In [8]:
# Plot the hotels on top of the humidity heatmap with each pin containing 
# the **Hotel Name**, **City**, and **Country**.
locations = hotel_df[["Latitude", "Longitude"]].astype(float)

# Info box template
template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Put hotel info in template
hotel_info = [template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, scale=2, fill_color='red', stroke_color='red',
    info_box_content=hotel_info)


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heatmap)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))